In [33]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [7]:
all_files = ['../../../datos/july_23/2023-07-01.csv', "../../../datos/july_23/2023-07-02.csv", '../../../datos/july_23/2023-07-03.csv', "../../../datos/july_23/2023-07-04.csv"]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [8]:
(df["from_address"] == df["to_address"]).sum()

7857

In [9]:
df = df[df["from_address"] != df["to_address"]]

In [10]:
(df["from_address"] == df["to_address"]).sum()

0

In [11]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [12]:
len(set(df['from_address']).union(set(df['to_address'])))

968950

In [13]:
df = df[~df.to_address.isna()]
df = df[~df.from_address.isna()]

In [14]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [15]:
ids

{'0x7771ed80f316bf5c3b22365849007d3e5d36eaf1': 0,
 '0x7d4dd242db341585021b180b710d4d67ccebb8c7': 1,
 '0x39ff42fff46fbb1426b4ebe20271e957a5a17710': 2,
 '0x5cd5085114f261d9526826dfe7ef956fd2a36634': 3,
 '0xfca3fbec024287060afef537f3560e7a83af947e': 4,
 '0x8a2b0e44f91d6fdc819225c61d674bf0ce44af76': 5,
 '0x4c2c4325ab7fb8775b7557df8c441091b6be0fe8': 6,
 '0xb77bcc5fd8986abca623afa8f47383d50a721775': 7,
 '0x536524308e1d8b6cd9acc0a6a1acd7d29a650a21': 8,
 '0x180935d280b55b6cd0569cbe94d72ef2a704a6e7': 9,
 '0xf8652fa13fe83df3d85bd3f0f4cf83470e644a3f': 10,
 '0x207e48866ff4885d024fbc2b01fdd2e1d7e1544c': 11,
 '0xe034fc3cc06c0f70c834a48c45a420c7fd0965a3': 12,
 '0x5c526a34d7595c5866872483237ec98bb67abd9f': 13,
 '0x4ad73170867953d9e0fabd87738d2009fd6b201d': 14,
 '0x0a548128048071decf20d7e4173bec452503948f': 15,
 '0x134e2c8cf875d5bcd4cc37007e350b843d760176': 16,
 '0x53b54a24b51ffcd41ecbf0d1b08e1686447c77fa': 17,
 '0x0e37ab2d2560d51595e5fdec5e80c3235df0298e': 18,
 '0x3e56f83622fcc0eff8aee7ec25a7abb69175e

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            124023936 
                                                                 
 dense (Dense)               (None, 1, 64)             8256      
                                                                 
 batch_normalization (Batch  (None, 1, 64)             256       
 Normalization)                                                  
                                                                 
Total params: 124032448 (473.15 MB)
Trainable params: 124032320 (473.15 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [17]:
# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [18]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        init = index * self.batch_size
        end = (index + 1) * self.batch_size

        # el batch van a ser #batch_size tuplas que representan el anchor y positive. El negative es un random.
        batch = self.df[init:end]

        # me agarro un sample de #batch_size transacciones.
        negative = self.df.sample(len(batch))

        anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
        positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
        negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
        
        anchor = tf.convert_to_tensor(anchor)
        positive = tf.convert_to_tensor(positive)
        negative = tf.convert_to_tensor(negative)
        
        # el fake target simunla lo que sería aprendizaje supervisado        
        fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

        return ([anchor, positive, negative], [fake_target]*3)

In [19]:
generator = GeneratorTriplet(df, ids, 64)

In [20]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=0.2, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)
        distance_positive = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        distance_negative = tf.reduce_sum(tf.square(anchor - negative), axis=1)

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)
        return tf.reduce_mean(loss)

loss = TripletCustom()    

model.compile(
    optimizer=Adam(2e-4),
    loss=loss
)

In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, min_delta=0.06)

model.fit(generator, epochs=5, callbacks=[callback])

Epoch 1/5
30193/30193 [==============================] - 1332s 44ms/step - loss: 0.0641
Epoch 2/5
30193/30193 [==============================] - 1258s 42ms/step - loss: 0.0407
Epoch 3/5
30193/30193 [==============================] - 1191s 39ms/step - loss: 0.0373


### Construcción de la matriz de embeddings

In [ ]:
embedding_matrix = model_aux.get_layer(name="embedding").get_weights()


In [60]:
# Definir los nombres de los meses y el número de días en cada mes
meses = ["july"]
dias_por_mes = {"june": 30, "july": 31}
indice_por_mes = {"june": 6, "july": 7}

# Función para obtener los archivos de un directorio dado
def obtener_archivos(ruta_directorio):
    return [os.path.join(ruta_directorio, archivo) for archivo in os.listdir(ruta_directorio)]

# Función para obtener las fechas en una ventana temporal
def obtener_fechas_ventana(fecha_inicial, ventana):
    fechas = []
    for i in range(ventana):
        fecha = fecha_inicial + timedelta(days=i)
        fechas.append(fecha.strftime("%Y-%m-%d"))
    return fechas

# Construir el array deseado
array_datos = []

for year in range(2023, 2024):
    for mes in meses:
        ruta_mes = f"../../../datos/{year}/{mes}"
        archivos_mes = obtener_archivos(ruta_mes)
        array_mes = [[archivo] for archivo in archivos_mes]
        array_datos.extend(array_mes)

        for dia in range(1, dias_por_mes[mes] + 1):
            fecha_actual = datetime(year, indice_por_mes[mes], dia)
            
            # Ventana de una semana
            ventana_una_semana = obtener_fechas_ventana(fecha_actual, 7)
            archivos_semana = [os.path.join(ruta_mes, f"{fecha}.csv") for fecha in ventana_una_semana]
            array_datos.append(archivos_semana)

            # Ventana de quince días
            ventana_quince_dias = obtener_fechas_ventana(fecha_actual, 15)
            archivos_quince_dias = [os.path.join(ruta_mes, f"{fecha}.csv") for fecha in ventana_quince_dias]
            array_datos.append(archivos_quince_dias)

print(sorted(array_datos))
print(len(array_datos))

FileNotFoundError: [Errno 2] No such file or directory: '../../../datos/2023/july'

In [62]:
import os
from datetime import datetime, timedelta

# Definir los nombres de los meses y el número de días en cada mes
meses = ["July"]
dias_por_mes = {"June": 30, "July": 31}

# Función para obtener los archivos de un directorio dado
def obtener_archivos(ruta_directorio):
    return [os.path.join(ruta_directorio, archivo) for archivo in os.listdir(ruta_directorio)]

# Función para obtener las fechas en una ventana temporal
def obtener_fechas_ventana(fecha_inicial, ventana):
    fechas = []
    for i in range(0, dias_por_mes[fecha_inicial.strftime("%B")], ventana):
        ventana_inicio = fecha_inicial + timedelta(days=i)
        ventana_fin = min(ventana_inicio + timedelta(days=ventana - 1), datetime(fecha_inicial.year, fecha_inicial.month, dias_por_mes[fecha_inicial.strftime("%B")]))
        fechas.append((ventana_inicio.strftime("%Y-%m-%d"), ventana_fin.strftime("%Y-%m-%d")))
    return fechas

# Construir el array deseado
array_datos = []

for year in range(2023, 2024):  # ajustar los años según sea necesario
    for mes in meses:
        ruta_mes = f"../../../datos/{year}/{mes}"
        archivos_mes = obtener_archivos(ruta_mes)
        array_mes = [[archivo] for archivo in archivos_mes]
        array_datos.extend(array_mes)

        # Agregar la ventana del mes completo
        archivos_mes_completo = [os.path.join(ruta_mes, f"{year}-{mes}-{day}.csv") for day in range(1, dias_por_mes[mes] + 1)]
        array_datos.append(archivos_mes_completo)

        # Agregar las ventanas de 7 días
        for i in range(0, dias_por_mes[mes], 7):
            ventana_inicio = datetime(year, meses.index(mes) + 1, i + 1)
            ventana_fin = min(ventana_inicio + timedelta(days=6), datetime(year, meses.index(mes) + 1, dias_por_mes[mes]))

            # Asegurarnos de que los días estén dentro del mes actual
            archivos_ventana_7_dias = []
            for day in range(ventana_inicio.day, ventana_fin.day + 1):
                if day <= dias_por_mes[mes]:
                    archivos_ventana_7_dias.append(os.path.join(ruta_mes, f"{year}-{mes}-{day}.csv"))

            array_datos.append(archivos_ventana_7_dias)

        # Agregar las ventanas de 15 días
        for i in range(0, dias_por_mes[mes], 15):
            ventana_inicio = datetime(year, meses.index(mes) + 1, i + 1)
            ventana_fin = min(ventana_inicio + timedelta(days=14), datetime(year, meses.index(mes) + 1, dias_por_mes[mes]))

            # Asegurarnos de que los días estén dentro del mes actual y no se dupliquen
            archivos_ventana_15_dias = []
            for day in range(ventana_inicio.day, ventana_fin.day + 1):
                if day <= dias_por_mes[mes] and day > i:
                    archivos_ventana_15_dias.append(os.path.join(ruta_mes, f"{year}-{mes}-{day}.csv"))
            
            if len(archivos_ventana_15_dias) == 1:
                continue

            array_datos.append(archivos_ventana_15_dias)

print(array_datos)

[['../../../datos/2023/July/2023-07-02.csv'], ['../../../datos/2023/July/2023-07-08.csv'], ['../../../datos/2023/July/2023-07-05.csv'], ['../../../datos/2023/July/2023-07-18.csv'], ['../../../datos/2023/July/2023-07-28.csv'], ['../../../datos/2023/July/2023-07-11.csv'], ['../../../datos/2023/July/2023-07-13.csv'], ['../../../datos/2023/July/2023-07-10.csv'], ['../../../datos/2023/July/2023-07-25.csv'], ['../../../datos/2023/July/2023-07-16.csv'], ['../../../datos/2023/July/2023-07-21.csv'], ['../../../datos/2023/July/2023-07-22.csv'], ['../../../datos/2023/July/2023-07-29.csv'], ['../../../datos/2023/July/2023-07-09.csv'], ['../../../datos/2023/July/2023-07-19.csv'], ['../../../datos/2023/July/2023-07-04.csv'], ['../../../datos/2023/July/2023-07-20.csv'], ['../../../datos/2023/July/2023-07-12.csv'], ['../../../datos/2023/July/2023-07-14.csv'], ['../../../datos/2023/July/2023-07-07.csv'], ['../../../datos/2023/July/2023-07-27.csv'], ['../../../datos/2023/July/2023-07-03.csv'], ['../../.